In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
from dateutil.parser import parse

warnings.filterwarnings("ignore")

In [2]:
data = pd.read_excel("Coca-Cola_stock_history.xlsx")

def make_naive(x):
    try:
        dt = parse(str(x))
        return dt.replace(tzinfo=None) if dt.tzinfo else dt
    except Exception:
        return pd.NaT

data['Date'] = data['Date'].apply(make_naive)
data.dropna(subset=['Date'], inplace=True)
data.set_index('Date', inplace=True)

info = pd.read_excel("Coca-Cola_stock_info.xlsx", header=None, names=["Description", "Information"])
info.dropna(inplace=True)
info = info[info['Information'] != 'nan']
info.reset_index(drop=True, inplace=True)

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
data['MA_20'] = data['Close'].rolling(window=20).mean()
data['MA_50'] = data['Close'].rolling(window=50).mean()
data['Daily_Return'] = data['Close'].pct_change()
data['Volatility'] = data['Daily_Return'].rolling(window=20).std()
data.dropna(inplace=True)

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(data['Close'], label='Close')
plt.plot(data['MA_20'], label='MA 20')
plt.plot(data['MA_50'], label='MA 50')
plt.legend()
plt.title("Coca-Cola Stock Price with Moving Averages")
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
features = ['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits',
            'MA_20', 'MA_50', 'Daily_Return', 'Volatility']
X = data[features]
y = data['Close']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("📉 Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("📉 Mean Absolute Error:", mean_absolute_error(y_test, y_pred))

In [ ]:
ticker = 'KO'
live = yf.download(ticker, period='1d', interval='1m')

live['MA_20'] = live['Close'].rolling(window=20).mean()
live['MA_50'] = live['Close'].rolling(window=50).mean()
live['Daily_Return'] = live['Close'].pct_change()
live['Volatility'] = live['Daily_Return'].rolling(window=20).std()

live['Dividends'] = 0.0
live['Stock Splits'] = 0.0

live.fillna(0, inplace=True)

features = ['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits', 
            'MA_20', 'MA_50', 'Daily_Return', 'Volatility']

latest = live[features].iloc[-1:].dropna()

if not latest.empty:
    live_pred = model.predict(latest)
    print(f"💡 Live Predicted Close Price: ${live_pred[0]:.2f}")
else:
    print("⚠️ Not enough live data available to make a prediction. Try again later.")

In [ ]:
print("\n🔎 Key Coca-Cola Company Metrics:")
for term in ['MarketCap', 'DividendYield', 'ProfitMargins', 'RevenueGrowth', 'Beta', 'ForwardPE']:
    match = info[info["Description"].str.contains(term, case=False)]
    if not match.empty:
        print(f"{match.iloc[0]['Description']}: {match.iloc[0]['Information']}")